In [1]:
import requests
import tarfile
import io
from io import BytesIO
import re
import os
import json
import fitz
import ast
import time


In [6]:
from dotenv import load_dotenv
load_dotenv('Key.env')

True

In [7]:
from openai import OpenAI
client = OpenAI(
    #Creates a client class
        api_key=os.environ.get("API_KEY"),
    )

In [8]:
from bs4 import BeautifulSoup

url = "https://arxiv.org/search/?query=large+language+models&searchtype=all&source=header"

response = requests.get(url)
pdf_source_links = []

# Check if the request was successful
# Scrapes first 50 LLM papers from the URL above
if response.status_code != 200:
    raise Exception("Request Failed!")

soup = BeautifulSoup(response.content, "html.parser")
links = [a.get('href') for a in soup.find_all('a', href=True)]

for link in links:
    if link.startswith("https://arxiv.org/pdf"):
        pdf_source_links.append(link)

In [9]:
def is_valid_python_code(code_str):
    try:
        ast.parse(code_str)
        return True
    except SyntaxError:
        return False

In [13]:
def process_pdf(url):
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception("Request Failed!")

    # Open the PDF file with PyMuPDF directly from the in-memory bytes
    pdf_document = fitz.open(stream=BytesIO(response.content), filetype='pdf')

    # Extract text from each page and store it in a string
    pdf_text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        pdf_text += page.get_text()

    # Optionally, encode the string as UTF-8
    pdf_text_utf8 = pdf_text.encode('utf-8').decode('utf-8')

    # Close the PDF document
    pdf_document.close()
    return pdf_text_utf8

def grab_authors(pdf):
    #no good solution, simply grabs the first few hundred characters and hopes all the authors are in that area, has already failed so far, let's improve on this later
    intro = pdf[:1500]
    #prompts gpt [possibly just hard code this solutoin, honestly just doing this for practice]

    valid_syntax = True
    loop_count = 3
    while(valid_syntax):
        loop_count -= 1
        if(loop_count == 0):
          #To prevent an infinite loop
          return False

        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": "Without any extra explanation text, find all authors and the organizations they work for, then sort them into a dictionary" + intro,
                }
            ],
            model="gpt-3.5-turbo",
        )

        valid_syntax = not is_valid_python_code(chat_completion.choices[0].message.content)

    #AST will automatically throw an error if GPT's response is deemed bad
    return ast.literal_eval(chat_completion.choices[0].message.content)

In [14]:
json_file_path = 'Arxiv_demo.json'

with open(json_file_path, 'r') as file:
    data = json.load(file)

for link in pdf_source_links:

    start_time = time.time()

    pdf_text = process_pdf(link)
    authors = grab_authors(pdf_text)

    #Failsafe so a batch won't get ruined
    if not authors:
      print("Failed! " + link[22:])
      continue
    
    #add to json
    data[link[22:]] = {"authors": authors}

    #confirmation of functionality
    end_time = time.time()
    print("Success! " + link[22:] + " "  + str(end_time - start_time))

with open(json_file_path, 'w') as file:
    json.dump(data, file, indent=4)

Success! 2408.16769 2.1384806632995605
Success! 2408.16756 2.0002357959747314
Success! 2408.16753 0.7691285610198975
Success! 2408.16749 2.628912925720215
Success! 2408.16740 0.6930596828460693
Success! 2408.16730 2.818835973739624
Success! 2408.16700 11.638014316558838
Success! 2408.16673 1.9014201164245605
Success! 2408.16612 4.120687246322632
Success! 2408.16601 2.4146924018859863
Success! 2408.16586 0.8991751670837402
Success! 2408.16518 2.26721453666687
Success! 2408.16502 2.1832878589630127
Success! 2408.16498 2.286269187927246
Success! 2408.16482 1.1681289672851562
Success! 2408.16465 4.541899681091309
Success! 2408.16440 0.7185852527618408
Success! 2408.16423 1.816413402557373
Success! 2408.16400 1.45489501953125
Success! 2408.16357 2.610882043838501
Success! 2408.16345 1.8995857238769531
Success! 2408.16326 1.8486905097961426
Success! 2408.16312 1.3202803134918213
Success! 2408.16304 1.8989272117614746
Success! 2408.16296 2.1751792430877686
Success! 2408.16276 0.82468008995056

In [16]:
import sqlite3

connection = sqlite3.connect('paper-table')

cursor = connection.cursor()

cursor.execute